# Open ETABS

In [88]:
import os
import sys
import comtypes.client
import openpyxl

#set the following flag to True to attach to an existing instance of the program
#otherwise a new instance of the program will be started
AttachToInstance = False

#set the following flag to True to manually specify the path to ETABS.exe
#this allows for a connection to a version of ETABS other than the latest installation
#otherwise the latest installed version of ETABS will be launched
SpecifyPath = True

#if the above flag is set to True, specify the path to ETABS below
ProgramPath = "C:\Program Files\Computers and Structures\ETABS 17\ETABS.exe"

#full path to the model
#set it to the desired path of your model
APIPath = r"C:\Users\mcoar\Documents\P1293\P1293_Lagos\ETABS\V2.0"
if not os.path.exists(APIPath):
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
FileName = 'P1293-B_V2.0.edb'
ModelPath = APIPath + os.sep + FileName

if AttachToInstance:
    #attach to a running instance of ETABS
    try:
        #get the active ETABS object
        myETABSObject = comtypes.client.GetActiveObject("CSI.ETABS.API.ETABSObject") 
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)

else:
    #create API helper object
    helper = comtypes.client.CreateObject('ETABS2016.Helper')
    helper = helper.QueryInterface(comtypes.gen.ETABS2016.cHelper)
    if SpecifyPath:
        try:
            #'create an instance of the ETABS object from the specified path
            myETABSObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:

        try: 
            #create an instance of the ETABS object from the latest installed ETABS
            myETABSObject = helper.CreateObjectProgID("CSI.ETABS.API.ETABSObject") 
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)

    #start ETABS application
    myETABSObject.ApplicationStart()

# Open Model

In [89]:
#create SapModel object
SapModel = myETABSObject.SapModel

#initialize model
SapModel.InitializeNewModel()

#open an existing file 
ret = SapModel.File.OpenFile(ModelPath)

#switch to k-in units
# kip_in_F = 3
# ret = SapModel.SetPresentUnits(kip_in_F)


# Run Model

In [90]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

# Test Bed

## Retrieve the names of all frames connected to the story LO1.

In [91]:
StoryName = 'L01'
NumberNames = 0
MyName = []
[NumberNames, MyName, ret] = SapModel.FrameObj.GetNameListOnStory(StoryName,NumberNames,MyName)
print(NumberNames)

96


## Retrieve the names of all frames (i.e., columns) in the user-defined group "BaseColumn"

In [92]:
GroupName = "BaseColumn"
NumberItems = 0  # initialize as double
ObjectType = []  # initialize as empty list
ObjectName = []  # initialize as empty list
[NumberItems, ObjectType, ObjectName, ret] = SapModel.GroupDef.GetAssignments(GroupName,NumberItems,ObjectType,ObjectName)
print(NumberItems)
    

38


## Retrieve the forces in the columns in the user-defined group "BaseColumn"

In [93]:
# deselect all cases and combos
ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
# set case selected for output
ret = SapModel.Results.Setup.SetCaseSelectedForOutput("CDL")
# initialize variable names
GroupName = "BaseColumn"
ItemType = 2      # eItemTypeElm.2 = a group item
NumberResults = 0
Obj = []; ObjSta = []
Elm = []; ElmSta = []
LoadCase = ['CDL']
StepType = []; StepNum = []
P = []
V2 = []; V3 = []
T = []
M2 = []; M3 = []

[NumberResults, Obj, ObjSta, Elm, ElmSta, LoadCase, StepType, StepNum, P, V2, V3, T, M2, M3, ret] = \
SapModel.Results.FrameForce("BaseColumn", \
                            ItemType, NumberResults, Obj, ObjSta, Elm, ElmSta, LoadCase, StepType, StepNum, \
                            P, V2, V3, T, M2, M3)

# Close ETABS

In [94]:
#save model
ret = SapModel.File.Save(ModelPath)

#close the program
ret = myETABSObject.ApplicationExit(False)
SapModel = None
myETABSObject = None

# Define Column Class

In [113]:
class Col:
    
    level = 'L01'
    
    def __init__(self):
        self.ID = []
        self.Pu = []     # factored load on column [kip]
        self.fc = []     # concrete strength of PILE [psi]
        self.Np = []     # Number of piles
        self.Pn = []     # Nominal capacity [kip]
        self.c = []    # column square dimensions [in]
        self.D = []      # Pilecap depth [in]
        self.dp = []     # pile diameter [in]
        self.spacing =[] # spacing between piles [in]
        self.cover =  [] # minimum concrete cover [in]
        self.dc =     [] # pile embedment [in]
        self.E =      [] # edge distance [in]
        self.NreinA = [] # Number of reinforcing bars in the long direction
        self.SreinA = [] # Size   of reinforcing bars in the long direction
        self.NreinB = [] # Number of reinforcing bars in the short direction
        self.SreinB = [] # Size   of reinforcing bars in the short direction

    def SelectCap(self, fc = 3, T = 40, maxdim = 30, maxD = 100):
        """
        This method takes an object of class Col and chooses a tabulated column pile cap
        design with maximum dimensions -maxdim- [ft] and max depth -maxD- [in]. Concrete 
        strength -fc- [ksi] and unfactored capacity -T- [tons]. 
        """
        import xlwings as xw
        import time
        
        t = time.time()
        app = xw.App(visible = False)
        wb = app.books.open('H:\Design Spreadsheets\CRSIDesignTables\PileCapTable.xlsx')
        topen = time.time()-t
        
        t = time.time()
        sn = str(fc)+'ksi-'+str(T)+'T'
        sheet = wb.sheets[sn]
        Nrows = sheet.range('A1').current_region.last_cell.row
        begin = 8                                 # Header takes up 7 rows

        
        for x in range(begin,Nrows+1):
#             print("{}:{}".format(x,sheet.range(x,2).value))
            try:
                if sheet.range(x,2).value > abs(self.Pu): # Need to add checks for long and short dimensions
                    Choice = x
#                     print("Good! Pn = {0:4.0f} kip is greater than Pu = {1:4.0f} kip".format(sheet.range(x,2).value, abs(self.Pu)))
                    break
                elif x == Nrows: # eventually need to add an option here to go to next worksheet if none of these work
                    Choice = x
#                     print("No good! but we haven't implemented a switch to the next worksheet, so we're just gonna \n say this works until we do")
                else: 
#                     print("No good: Pn = {0:4.0f} kip must be greater than Pu = {1:4.0f} kip".format(sheet.range(x,2).value, abs(self.Pu)))
                    continue
            except:
#                     print('Likely a blank entry on the ')   
                continue
        tchoose = time.time() - t
        
        t = time.time()
        self.Np = sheet.range(Choice,1).value       # Number of piles
        self.Pn = abs(sheet.range(Choice,2).value)   # Nominal capacity [kip]
        self.c = sheet.range(Choice,3).value     # column square dimensions [in]
        self.D = sheet.range(Choice,8).value        # Pilecap depth [in]

        self.dp = sheet.range(2,4).value          # pile diameter [in]
        self.spacing = sheet.range(2, 5).value    # spacing between piles [in]
        self.cover = sheet.range(2,6).value       # minimum concrete cover [in]
        self.dc = sheet.range(2, 7).value         # pile embedment [in]
        self.E = sheet.range(2,8).value           # edge distance [in]

        self.NreinA = sheet.range(Choice,10).value # Number of reinforcing bars in the long direction
        self.SreinA = sheet.range(Choice,11).value # Size   of reinforcing bars in the long direction
        self.NreinB = sheet.range(Choice,13).value # Number of reinforcing bars in the short direction
        self.SreinB = sheet.range(Choice,14).value # Size   of reinforcing bars in the short direction
        
        tassign = time.time()-t
        print("{0}: {1:1.5f} {2:1.5f} {3:1.5f}".format(self.ID,topen,tchoose,tassign))
        
        
    def popxlsx(self,pn,fn):
        """
        This method takes an instance of class Col() that has been assigned pile cap design parameters
        based on method SelectCap() and populates a design workbook with these parameters for engineer
        inspection.
        """
        import xlwings as xw
        ft = '.xlsx'                         # filetype
        app = xw.App(visible = False)
        wb = app.books.open(pn+fn+ft) # open workbook
        ws = wb.sheets['DESIGN CRIT'] # make this sheet active

        # Update sheet based on column parameters
        ws.range('F45').value  = self.Np
        ws.range('E30').value  = self.c
        ws.range('E27').value  = self.D 
        ws.range('E29').value  = self.dp
        ws.range('E103').value = self.spacing / 12 
        ws.range('E38').value  = self.cover
        ws.range('E31').value  = self.dc
        ws.range('E28').value  = self.E
        ws.range('E353').value = self.NreinA
        ws.range('F353').value = '#'+str(self.SreinA)
        ws.range('E387').value = self.NreinB
        ws.range('F387').value = '#'+str(self.SreinB)              

        wb.save(pn+fn+'_'+self.ID+ft) # save workbook
        wb.close()


# Select Tabulated PileCap Design

In [114]:
# Create a dictionary of Column Names and Internal Forces
columns = {}
InternalForces = {}

for i in range(0,len(Obj),3):
    columns["C"+Obj[i]] = Obj[i]
    InternalForces["C"+Obj[i]] = P[i]
    
# Replace the entries in the column dictionary with instances of class Col
print('Column - t_open - t_choose - t_assign')
for items in columns:
    columns[items] = Col()
    columns[items].ID = items
    columns[items].fc = 3
    columns[items].Pu = InternalForces[items]
    columns[items].SelectCap()

Column - t_open - t_choose - t_assign
C1: 16.63895 9.72656 7.47043
Column - t_open - t_choose - t_assign
C2: 22.19427 10.27659 9.51454
Column - t_open - t_choose - t_assign
C3: 19.34911 11.14964 11.32965
Column - t_open - t_choose - t_assign
C4: 23.91737 13.57778 10.42760
Column - t_open - t_choose - t_assign
C5: 23.77336 31.09178 5.07629
Column - t_open - t_choose - t_assign
C7: 23.24833 15.15187 5.24330
Column - t_open - t_choose - t_assign
C8: 23.50434 20.88919 5.37431
Column - t_open - t_choose - t_assign
C9: 22.63329 15.50489 13.33676
Column - t_open - t_choose - t_assign
C10: 24.87842 18.76407 10.95863
Column - t_open - t_choose - t_assign
C11: 25.40345 13.77579 11.47166
Column - t_open - t_choose - t_assign
C12: 23.40334 35.66004 5.59332



KeyboardInterrupt



# Populate Workbook

In [ ]:
### Load and Open Template

import xlwings as xw
pn = "H:\\Design Spreadsheets\\" # pathname
fn = "CRSI_08_Pilecap_Design"  # filename
ft = ".xlsx"                   # filetype
wb = xw.Book(pn+fn+ft) # open workbook
modelname = FileName[:-4]
newpn = pn+modelname+'\\' # pathname
newfn = fn+'_'+modelname

### Create Directory for PileCap Design Aids

try: 
    os.mkdir(newpn)
except FileExistsError:
    print('directory already exists: overwriting')
wb.save(newpn+newfn+ft) # save copy of template with new name specific to ETABS model

### Populate Sheets with Design

# loop through columns dictionary and populate an excel design aid with tabulated values for engineer inspection
for items in columns:
    columns[items].popxlsx(newpn,newfn)

In [ ]:
type(columns['C1'].spacing)